In [1]:
import plotsettings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.special as ss
import plotsettings

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %load_ext autoreload

# %autoreload 0

#### General definitions:
$\mu(\beta) = \dfrac{m_W^2}{2 \, \beta}\,,$ where $\beta$ is magnetic field strength in $\mathrm{GeV}^2;$

$n_\mathrm{max}(x, \mu, m) = \left(\sqrt{x} - \sqrt{m}\right)^2 - \mu+\dfrac{1}{2};$

$m_\mathrm{max}(x, \mu, n) = \left(\sqrt{x} - \sqrt{n + \mu - \dfrac{1}{2}}\right)^2;$

$d(x, \mu, n, m) = \sqrt{ \left(x - \mu - n + m + \dfrac{1}{2}\right)^2 - 4 \, m \, x };$

In [3]:
m_W = mW = 80.385     # W-boson mass in GeV.
G_F = 1.16637E-05     # Fermi coupling constant in GeV^{-2}
Kfp = G_F * m_W * m_W * m_W / (2 * math.sqrt(2) * math.pi) # the common factor of the probability

In [4]:
floor = math.floor
sqrt = math.sqrt
exp = math.exp
isfinite = math.isfinite
# floor = np.floor
# sqrt = np.sqrt
# exp = np.exp

def mu(beta):
    """
    m_W^2 / (2 \beta);
    here beta is dimensionless parameter in units of m_W^2.
    """
    return 0.5 / beta

def n_max(x, mu, m=0):
    """
    n_max = (\sqrt{x} - \sqrt{m})^2 - \mu + 1/2
    """
    if m == 0:
        return floor(x-mu+0.5)
    else:
        t = sqrt(x) - sqrt(m)
        return floor(t*t - mu + 0.5)

def m_max(x, mu, n=0):
    """
    m_max = (\sqrt{x} - \sqrt{n + \mu - 1/2})^2
    """
    t = sqrt(x) - sqrt(n+mu-0.5)
    return floor(t*t)

def d(x, mu, n, m):
    """
    $d = \sqrt{ \left(x - \mu - n + m + 1/2\right)^2 - 4 \, m \, x };$
    """
    t = x - mu - n + m + 0.5
    return sqrt(t*t - 4*m*x)

In [39]:
copysign = math.copysign
sign = lambda x: copysign(1.0, x)

def heaviside_theta(x):
    """
    Def. of Heaviside theta function
    """
    return 0.5 * (copysign(1.0, x) + 1.0)

theta = heaviside_theta
laguerre_l = ss.eval_genlaguerre

def pow_multifactorial(x, a, b):
    """
    """
    if a == b:
        return 1.0
#         return 1.0/x
    if a < 0:
        if b < 0:
            return float('nan')
        else:
            return float('inf')
    elif b < 0:
        return 0.0

    s = 1.0
#     s = 1.0/x
    if a >= b:
        for i in range(b+1, a+1):
            s *= i/x
    else:
        for i in range(a+1, b+1):
            s *= x/i
    return s

def wp_nm(x, mu):
    """
    """
    s = 0.0

    nmax = n_max(x, mu)
    for n in range(nmax+1):
        mmax = m_max(x, mu, n)
        for m in range(mmax+1):
            if  (sqrt(x) - sqrt(n + mu - 0.5) - sqrt(m) >= 0):
                p0 = pow_multifactorial(x, m, n-1)
                p1 = pow_multifactorial(x, m, n)
                p2 = pow_multifactorial(x, m-1, n-2)
                if isfinite(p2) and (n > 1):
                    l2 = laguerre_l(m-1, n-m-1, x)
                    l2 = p2 * l2 * l2
                else:
                    l2 = 0
                l1 = laguerre_l(m, n-m, x)
                l1 = p1 * l1 * l1
                # theta(sqrt(x) - sqrt(n + mu - 0.5) - sqrt(m)) *
                s += (
                    -4 * p0 * x * laguerre_l(m, n-m-1, x) * laguerre_l(m-1, n-m, x) +
                    (x - mu - n + m + 0.5) * (l1 + l2)
                ) / d(x, mu, n, m)
    
    return Kfp*exp(-x)/sqrt(mu*x) * s

def wp_mn(x, mu):
    """
    """
    s = 0.0

    mmax = m_max(x, mu)
    for m in range(mmax+1):
        nmax = n_max(x, mu, m)
        for n in range(nmax+1):
            if (sqrt(x) - sqrt(n + mu - 0.5) - sqrt(m) >= 0):
                p0 = pow_multifactorial(x, n-1, m-1)
                p1 = pow_multifactorial(x, n, m)
                p2 = pow_multifactorial(x, n-2, m-1)
                if isfinite(p2) and (n > 1):
                    l2 = laguerre_l(n-2, m-n+1, x)
                    l2 = p2 * l2 * l2
                else:
                    l2 = 0
                l1 = laguerre_l(n, m-n, x)
                l1 = p1 * l1 * l1
                # theta(sqrt(x) - sqrt(n + mu - 0.5) - sqrt(m)) * 
                s += (
                    4 * p0 * x * laguerre_l(n-1, m-n+1, x) * laguerre_l(n-1, m-n, x) +
                    (x - mu - n + m + 0.5) * (l1 + l2)
                ) / d(x, mu, n, m)
    
    return Kfp*exp(-x)/sqrt(mu*x) * s

def wm(x, mu):
    """
    """
    pass


In [40]:
print(wp_mn(5.15, mu(1/10)))
print("?")
print(wp_nm(5.15, mu(1/10)))

0 0
0.000779222213005
?
0 0
nan


In [42]:
laguerre_l(-1, 0, 5.15)

0.0